In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Load the data
df = pd.read_csv('./processedData.csv')

# Parse the 'Full date' column and set it as index
df['Full date'] = pd.to_datetime(df['Full date'])
df.set_index('Full date', inplace=True)

# Feature Engineering: Extract features from the date
df['DayOfYear'] = df.index.dayofyear
df['Month'] = df.index.month
df['Day'] = df.index.day
df['Year'] = df.index.year

# Normalize the target variable (Temp Max)
scaler = MinMaxScaler(feature_range=(0, 1))
df['Temp Max'] = scaler.fit_transform(df['Temp Max'].values.reshape(-1, 1))
data = df[['Temp Max']].values
data 

Using device: cuda


array([[0.37515034],
       [0.33306613],
       [0.31022045],
       ...,
       [0.3470942 ],
       [0.34228462],
       [0.4       ]])

In [8]:
train_size = int(len(data) * 0.8)
train, test = data[:train_size], data[train_size:]

# Function to create a dataset suitable for LSTM
def create_dataset(dataset, time_step=1):
    X, y = [], []
    for i in range(len(dataset) - time_step - 1):
        X.append(dataset[i:(i + time_step), 0])
        y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(y)

# Create dataset with a reasonable time_step
time_step = 30  # Adjust as needed
X_train, y_train = create_dataset(train, time_step)
X_test, y_test = create_dataset(test, time_step)

In [9]:
X_train[0],y_train[0]

(array([0.37515034, 0.33306613, 0.31022045, 0.32424852, 0.35030061,
        0.35871748, 0.31462929, 0.33787578, 0.37755513, 0.38156316,
        0.3915832 , 0.38597193, 0.42164332, 0.40320641, 0.39478961,
        0.37675351, 0.40601201, 0.41322645, 0.438477  , 0.43687375,
        0.43206418, 0.43246499, 0.40961923, 0.42124251, 0.43086174,
        0.43607213, 0.43166336, 0.45410823, 0.44368738, 0.41122247]),
 0.4280561442077757)